# **Assignment 2 ROB313**

In [3]:
# Import revelant libraries

import numpy as np
import matplotlib.pyplot as plt
from time import time
from data.data_utils import load_dataset
from sklearn.neighbors import KDTree
from scipy.linalg import cho_factor, cho_solve
np.random.seed(21)

 **Question 3 - Creating a Radial Basis Function (RBF)**

In [14]:
def gaussian_kernel(x_train, x_test, theta):
    # reshape the matricies correctly for broadcasting
    x = np.expand_dims(x_train, axis=1)
    z = np.expand_dims(x_test, axis=0)
    # now evaluate the kernel using the euclidean distances squared between points
    return np.exp(-np.sum(np.square(x-z)/theta, axis=2, keepdims=False))

def rbf_model(x_train, y_train, x_valid, y_valid):
    '''Format data, determine alpha by calculating gram matrix and usng Cholesky factorization, apply RBF model, iterate over values of theta and lamda and store RMSE info'''
    thet = np.array([0.05, 0.1, 0.5, 1, 2])
    lam = np.array([0.001, 0.01, 0.1, 1])
    rmse = np.zeros((thet.shape[0], lam.shape[0]))
    # Select hyperparameters by evaluating on the validation set
    for i,theta in enumerate(thet):
        for j,lamda in enumerate(lam):
            K = gaussian_kernel(x_train, x_train, theta)
            C = cho_factor(K + lamda*np.identity(x_train.shape[0]))
            alpha = cho_solve(C,y_train)
            # Use x_train as the landmarks (basis vectors) for predicting on validation set
            y_rbf = gaussian_kernel(x_valid, x_train, theta).dot(alpha)
            rmse[i][j] = np.sqrt(np.mean(np.square(y_rbf-y_valid)))
    # Determine hyperparameters with smallest error
    min_ind = np.unravel_index(np.argmin(rmse, axis=None), rmse.shape)
    theta_best = thet[min_ind[0]]
    lamda_best = lam[min_ind[1]]

    return theta_best, lamda_best

def test_rbf(x_train, y_train, x_valid, y_valid, x_test, y_test, theta, lamda):
    # Use training and validation sets to predict on test set
    x_train = np.vstack([x_train, x_valid])
    y_train = np.vstack([y_train, y_valid])

    # Calculate RBF using hyperparameters given
    K = gaussian_kernel(x_train, x_train, theta)
    C = cho_factor(K + lamda*np.identity(x_train.shape[0]))
    alpha = cho_solve(C,y_train)
    # Use x_train as the landmarks (basis vectors) for predicting on test set
    y_rbf = gaussian_kernel(x_test, x_train, theta).dot(alpha)
    rmse = np.sqrt(np.mean(np.square(y_rbf-y_test)))

    return rmse
    
def run_Q3(dataset):
    if dataset == 'mauna_loa':
        x_train, x_valid, x_test, y_train, y_valid, y_test = load_dataset('mauna_loa')
    elif dataset == 'rosenbrock':
        x_train, x_valid, x_test, y_train, y_valid, y_test = load_dataset('rosenbrock', n_train=1000, d=2)

    # Determine hyperparameters (regularization and shape) using training and validation sets
    theta, lamda = rbf_model(x_train, y_train, x_valid, y_valid)
    # Assess hyperparameter choices against test data
    test_rmse = test_rbf(x_train, y_train, x_valid, y_valid, x_test, y_test, theta, lamda)

    return test_rmse, theta, lamda

run_Q3('rosenbrock')

(0.14812442755080485, 2.0, 0.001)

**Results:**

Mauna Loa:
- Test RMSE: 0.14977338771865478
- Theta: 1.0
- Lamda = 0.001

Rosenbrock:
- Test RMSE: 0.14812442755080485
- Theta: 2.0
- Lamda: 0.001

 **Question 4 - Implementing Greedy Regression Algorithm**

In [ ]:
def ortho_match_pursuit(x_train, x_valid, x_test, y_train, y_valid, y_test):
    '''Create basis dictionary, use algorithm to determine which basis functions minimize the loss, then use those basis functions and weights from algo as our final sparse model to predict on'''
    pass

def run_Q4():
    x_train, x_valid, x_test, y_train, y_valid, y_test = load_dataset('mauna_loa')
    ortho_match_pursuit(x_train, x_valid, x_test, y_train, y_valid, y_test)

